# **Важно!**

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** на учебной платформе в **стандартную форму сдачи домашнего задания.**

**Срок проверки преподавателем:** домашнее задание проверяется **в течение 3 дней после дедлайна сдачи** с предоставлением обратной связи

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

После ячеек с заданием следуют ячейки с проверкой **с помощью assert.**

Если в коде есть ошибки, assert выведет уведомление об ошибке.

Если в коде нет ошибок, assert отработает без вывода дополнительной информации.

---

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Упрощенная модель GPT

### Конфигурация модели

In [ ]:
config = {
  "n_ctx": 32,  # Максимальная длина контекста (context).
  "n_embd": 64,  # Размерность встраивания (embedding dimension).
  "n_head": 4,  # Количество внимательных голов (attention heads) в многоголовом внимании.
  "n_layer": 4,  # Количество слоев в модели.
  "dropout": 0.0,  # Дропаут  (dropout) для предотвращения переобучения.
  "vocab_size": None,  # Размер словаря модели.
  "batch_size": 16,  # Размер словаря модели.
  "eval_interval": 100,  # Шаг для вывода оценки модели.
  "eval_iters": 100,  # Шаг для вывода оценки модели.
  "learning_rate": 1e-3,  # Шаг обучения.
}
config["head_size"]  = config["n_embd"] // config["n_head"]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

### Подготовка данных

Процесс подготовки данных включает в себя следующие этапы:

1. Сбор данных: Сначала необходимо собрать исходные данные, на которых будет обучаться модель GPT. Это может быть большой корпус текстов, содержащий разнообразные данные, в зависимости от вашей задачных: Данные обычно требуют предварительной обработки, включая очистку от мусорных символов, токенизацию (разбиение текста на токены, как слова или подстр тру и другие операции, зависящие от задач2.

3. Токенизация: Модели GPT работают с токенами, поэтому текст должен быть преобразован в последовательность токенов. Вы можете использовать токенизаторы, предоставляемые библиотеками, такими как Hugging Face Transformers, для выполнения этой оп
   
3. Разделение данных: Данные обычно разделяются на тренировочную, валидационную и тестовую выборки для оценки и тестирования модели.ер43ии.

4. Создание последовательностей: Данные обычно разбиваются на последовательности или "окна", чтобы модель могла изучать контекст. Это может потребовать создания окон разной длины, в зависимости от задачи. Создание целевых последовательностей (для обучения с учителем): Если вы обучаете модель с учителем, вам также потребуется создать целевые последовательности, которые модель будет пытаться пр ответ на запросы.

### 1. Сбор данных

In [ ]:
# with open('data/input.txt', 'r', encoding='utf-8') as f:
#    text = f.read()
with open('/content/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

# let's look at the first 1000 characters
print(text[:100])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### 2. Токенизация для упрощенной модели GPT

In [ ]:
# Токенизация для упрощенной модели GPT
chars = sorted(list(set(text)))
config["vocab_size"] = len(chars)
print(''.join(chars))
print(config["vocab_size"])


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


### 3. Разделение данных

In [ ]:
# Train and test splits

# put all data into a PyTorch tensor
data = torch.tensor(encode(text), dtype=torch.long)

# creating train and test datasets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(train_data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


### 4. Создание последовательностей

In [ ]:
# printing a n_ctx of train data
n_ctx = config["n_ctx"]
batch_size = config["batch_size"]
train_data[:n_ctx +1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1])

In [ ]:
# exploring test and train data
x = train_data[:n_ctx]
y = train_data[1:n_ctx+1]
for t in range(n_ctx):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) the target: 64
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) the target: 43
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) the target: 52
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) the target: 10
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) the target: 0
when input is tensor([18, 47, 56, 57, 58,  1, 15, 

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - n_ctx, (batch_size,))
    x = torch.stack([data[i:i+n_ctx] for i in ix])
    y = torch.stack([data[i+1:i+n_ctx+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
# testing get_batch function

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(n_ctx): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        if b == 0:
            print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([16, 32])
tensor([[39, 52, 42,  1, 52, 53, 61, 12,  1, 61, 46, 43, 56, 43,  1, 40, 43,  1,
         58, 46, 63,  1, 40, 56, 53, 58, 46, 43, 56, 57, 12,  0],
        [ 6,  1, 57, 43, 43, 49,  6,  1, 39, 52, 42,  1, 49, 52, 53, 61,  1, 46,
         53, 61,  1, 58, 46, 47, 57,  1, 44, 53, 59, 50,  1, 51],
        [50,  1, 56, 39, 47, 50,  1, 47, 52,  0, 46, 47, 57,  1, 56, 53, 54, 43,
          7, 58, 56, 47, 41, 49, 57,  8,  1, 21,  5, 50, 50,  1],
        [39, 52, 42,  1, 43, 52, 58, 56, 43, 39, 58,  1, 58, 46, 43, 51,  6,  0,
         18, 53, 56,  1, 51, 63,  1, 61, 53, 59, 52, 42, 57,  5],
        [20, 43, 56, 43,  1, 41, 53, 51, 43, 57,  1, 30, 53, 51, 43, 53,  6,  1,
         46, 43, 56, 43,  1, 41, 53, 51, 43, 57,  1, 30, 53, 51],
        [57, 58,  1, 58, 46, 53, 59,  1, 44, 43, 43, 50,  1, 58, 46, 63, 57, 43,
         50, 44,  1, 52, 53, 61, 12,  0,  0, 31, 43, 41, 53, 52],
        [43,  2,  0, 16, 43, 39, 58, 46,  6,  1, 58, 46, 39, 58,  1, 46, 39, 58,
        

## Построение упрощенной модели GPT

In [ ]:
# loss function definition
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(config["eval_iters"])
        for k in range(config["eval_iters"]):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Attention блоки

Заметки о внимании от А. Карпатного:
- Внимание - это **механизм коммуникации**. Можно рассматривать как узлы в ориентированном графе, смотрящие друг на друга и агрегирующие информацию со взвешенной суммой от всех узлов, которые указывают на них, с весами, зависящими от данных.
- Здесь нет понятия пространства. Внимание просто действует по набору векторов. Вот почему нам нужно позиционно кодировать токены.
- Каждый пример в пакетном измерении, конечно, обрабатывается полностью независимо и никогда не "разговаривает" друг с другом
- В блоке внимания "encoder" просто удалите единственную строку, которая выполняет маскировку с помощью "tril", позволяя всем токенам взаимодействовать. Этот блок здесь называется блоком внимания "декодер", потому что он имеет треугольную маскировку и обычно используется в настройках авторегрессии, таких как языковое моделирование.
- "само-внимание" просто означает, что ключи и значения создаются из того же источника, что и запросы. При "перекрестном внимании" запросы по-прежнему генерируются из x, но ключи и значения поступают из какого-то другого внешнего источника (например, модуля кодирования).
- "Масштабированное" внимание дополнительно делит `wei` на 1/ sqrt(head_size). Это делает так, что когда входные данные Q, K являются единичной дисперсией, wei тоже будет единичной дисперсией, а Softmax останется рассеянным и не будет насыщать слишком сильно. Иллюстрация ниже.
  
`torch.tril` определяет нижнюю треугольную матрицу, используемую для создания маскировки. Код для модуля Multihead Attention следующий.

In [ ]:
class Head(nn.Module):
    """ Один блок внимания в механизме само-внимания (self-attention) """

    def __init__(self, **kwargs):
        super().__init__()

        # Устанавливаем атрибуты класса на основе переданных аргументов
        for key, value in kwargs.items():
            setattr(self, key, value)

        # Создаем линейные преобразователи для ключей (key), запросов (query) и значений (value)
        self.key = nn.Linear(self.n_embd, self.head_size, bias=False)
        self.query = nn.Linear(self.n_embd, self.head_size, bias=False)
        self.value = nn.Linear(self.n_embd, self.head_size, bias=False)

        # Регистрируем буфер 'tril', который содержит нижний треугольник матрицы
        # с единицами на главной диагонали
        self.register_buffer('tril', torch.tril(torch.ones(self.n_ctx, self.n_ctx)))

        # Создаем слой Dropout
        self.dropout = nn.Dropout(self.dropout)

    def forward(self, x):
        """
        Задание 1 (2 балла): Вам предоставлен код класса Head, который представляет собой один блок механизма само-внимания в трансформере.

        Вам необходимо реализовать метод forward, который выполняет операцию само-внимания.
        Метод должен принимать входной тензор x формы (B, T, C), где B - размер пакета, T - количество токенов и
        C - размерность эмбеддингов.
        Вам нужно выполнить следующие шаги:

        Преобразовать входной тензор x с помощью линейных преобразователей для ключей (сохранить в переменную k)
        и запросов (сохранить в переменную q) .
        Вычислить оценки внимания (affinities) с помощью матричного умножения ключей и запросов.
        Применить маску для маскирования верхнего треугольника матрицы оценок внимания.
        Применить функцию Softmax для получения весов внимания.
        Применить слой Dropout к весам внимания.
        Выполнить взвешенную агрегацию значений с использованием весов внимания (сохранить в переменную v).
        Реализуйте метод forward так, чтобы он возвращал результат операции само-внимания.
        """
        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        B, T, C = x.size()
        k = self.key(x)
        q = self.query(x)
        affinities = q @ k.transpose(-2, -1) * C**-0.5
        masked = affinities.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        attnw = F.softmax(masked, dim=-1)
        attnw = self.dropout(attnw)
        v = self.value(x)
        out = attnw @ v
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ
        return out  # Возвращаем результат

In [ ]:
class MultiHeadAttention(nn.Module):
    """ Несколько блоков механизма само-внимания, работающих параллельно """

    def __init__(self, **kwargs):
        super().__init__()

        # Устанавливаем атрибуты класса на основе переданных аргументов
        for key, value in kwargs.items():
            setattr(self, key, value)

        # Создаем список блоков Head (блоки механизма само-внимания) в количестве n_head
        self.heads = nn.ModuleList([Head(**kwargs) for _ in range(self.n_head)])

        # Создаем линейный проецирующий слой для объединения результатов от разных голов
        self.proj = nn.Linear(self.n_embd, self.n_embd)

        # Создаем слой Dropout
        self.dropout = nn.Dropout(self.dropout)

    def forward(self, x):
        # Для каждой головы механизма само-внимания выполняем операцию self-attention
        # и конкатенируем результаты

        """
        Задание 2 (1 балл): Вам предоставлен код класса MultiHeadAttention, который представляет собой блок многоголового механизма само-внимания в трансформере.
        Вам нужно реализовать метод forward, который выполняет операцию многоголового само-внимания.
        Метод должен принимать входной тензор x формы (B, T, C), где B - размер пакета, T - количество токенов и
        C - размерность эмбеддингов. Вам нужно выполнить следующие шаги:

        Передать входной тензор x через каждую голову многоголового механизма само-внимания.
        Конкатенировать результаты от каждой головы вдоль последней размерности.
        Пропустить объединенные результаты через линейный проецирующий слой.
        Применить слой Dropout к выходу проецирующего слоя.
        Реализуйте метод forward так, чтобы он возвращал итоговый результат операции многоголового само-внимания.
        """

        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        heads = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(heads))
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ

        return out  # Возвращаем итоговый результат

Следующая ячейка содержит реализацию блока GPT Transformer, как показано в оригинальной статье. Это модуль ** декодера **, хотя он больше похож на модуль ** кодера ** трансформатора. Исходная архитектура использует 12 таких блоков, наша реализация использует 4 блока по умолчанию.
<center>

![Alt text](block.png)

</center>

In [ ]:
class FeedFoward(nn.Module):
    """ Простой блок с линейным слоем, за которым следует нелинейное преобразование """

    def __init__(self, **kwargs):
        super().__init__()

        # Устанавливаем атрибуты класса на основе переданных аргументов
        for key, value in kwargs.items():
            setattr(self, key, value)

        # Создаем последовательный нейронный блок (нейронную сеть)
        self.net = nn.Sequential(
            nn.Linear(self.n_embd, 4 * self.n_embd),  # Линейный слой с увеличением размерности
            nn.ReLU(),  # Функция активации ReLU
            nn.Linear(4 * self.n_embd, self.n_embd),  # Линейный слой с уменьшением размерности
            nn.Dropout(self.dropout),  # Слой Dropout для регуляризации
        )

    def forward(self, x):
        # Пропускаем входной тензор через нейронный блок (нейронную сеть)

        """
        Задание 3 (1 балл):
        Вам предоставлен код класса FeedFoward, который представляет собой простой блок в трансформере.
        Ваша задача - написать метод forward для этого класса. Метод forward принимает входной тензор x и должен выполнить следующие шаги:

        Пропустить входной тензор через линейный слой с увеличением размерности в 4 раза.
        Применить функцию активации ReLU к результатам линейного слоя.
        Пропустить результаты через второй линейный слой с уменьшением размерности до исходной размерности.
        Применить слой Dropout для регуляризации данных.
        Вернуть результат после обработки.
        Напишите метод forward так, чтобы он выполнял описанные выше операции.
        """

        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        return self.net(x)
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ


In [ ]:
class Block(nn.Module):
    """ Блок трансформера: коммуникация, затем вычисления """

    def __init__(self, **kwargs):
        # n_embd: размерность векторных представлений, n_head: количество "голов" внимания
        super().__init__()

        # Устанавливаем атрибуты класса на основе переданных аргументов
        for key, value in kwargs.items():
            setattr(self, key, value)

        # Создаем блок многоголового механизма само-внимания (self-attention)
        self.sa = MultiHeadAttention(**kwargs)

        # Создаем блок с нейронной сетью для вычислений
        self.ffwd = FeedFoward(**kwargs)

        # Создаем слои нормализации после каждого блока
        self.ln1 = nn.LayerNorm(self.n_embd)
        self.ln2 = nn.LayerNorm(self.n_embd)

    def forward(self, x):
        """
        Задание 4 (2 балла):
        Вам предоставлен код класса Block, который представляет собой блок в трансформере.
        Ваша задача - написать метод forward для этого класса. Метод forward выполняет следующие операции:

        Проходит входной тензор через блок многоголового механизма само-внимания (self.sa) и добавляет результат к исходному тензору, а затем применяет слой нормализации.
        Проходит обновленный тензор через блок вычислений (self.ffwd), добавляет результат к предыдущему тензору и снова применяет слой нормализации.
        Возвращает итоговый тензор после всех операций.
        Ваше задание - реализовать метод forward так, чтобы он выполнял описанные выше операции.
        """
        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ

        return x  # Возвращаем итоговый тензор после всех операций


### Сборка языковой модели


Модели, которые назначают вероятности последовательностям слов, называются **языковыми моделями**. Начиная с задачи вычисления $\mathrm{P}(w \mid h)$, вероятности *слова* $w$ при заданной *истории* $h$, можно рассмотреть историю $h$ = "нагреватель такой горячий, что" и вероятность того, что следующее слово - "нагреватель"

$$
\mathrm{P}(\text{нагреватель} \mid \text{нагреватель такой горячий, что})
$$

и вычислить ее как относительные частоты: взять очень большой корпус, посчитать количество случаев, когда мы видим "нагреватель такой горячий, что",
и посчитать количество случаев, когда за ним следует "нагреватель". Однако это не всегда выполнимо. Обычно используются *наивные методы*, что подразумевает использование упрощений.

**Биграммная модель** легко реализуется, но является очень наивной. Например, она аппроксимирует вероятность слова $w_n$ при заданных предыдущих словах

$$
\mathrm{P}(w_n \mid w_1, w_2, \dots, w_{n−1})
$$

используя только условную вероятность предшествующего слова $P(w_n \mid w_{n−1})$. Другими словами, вместо вычисления вероятности

$$
\mathrm{P}(\text{нагреватель} \mid \text{основной нагреватель такой горячий, что})\,,
$$

мы аппроксимируем ее вероятностью

$$
\mathrm{P}(\text{нагреватель} \mid \text{что})\,.
$$

In [ ]:
class GPTModel(nn.Module):

    def __init__(self, **kwargs):
        super().__init__()

        # Устанавливаем атрибуты класса на основе переданных аргументов
        for key, value in kwargs.items():
            setattr(self, key, value)

        # Создаем таблицу для эмбеддингов токенов
        self.token_embedding_table = nn.Embedding(self.vocab_size, self.n_embd)

        # Создаем таблицу для позиционных эмбеддингов
        self.position_embedding_table = nn.Embedding(self.n_ctx, self.n_embd)

        # Создаем последовательность блоков трансформера
        self.blocks = nn.Sequential(*[Block(**kwargs) for _ in range(self.n_layer)])

        # Создаем слой нормализации после последнего блока
        self.ln_f = nn.LayerNorm(self.n_embd)

        # Создаем линейный слой для преобразования в логиты
        self.lm_head = nn.Linear(self.n_embd, self.vocab_size)

    def forward(self, idx, targets=None):
        """
        Задание 5 (4 балла):
        Вам предоставлен код класса GPTModel, который представляет собой модель на основе архитектуры GPT (Generative Pre-trained Transformer).
        Ваша задача - реализовать метод forward для этого класса. Метод forward принимает входной тензор idx и, при необходимости, целевой тензор targets.

        Ваша задача - выполнять следующие операции в методе forward:

        Получить эмбеддинги токенов (tok_emb) из таблицы эмбеддингов для токенов.
        Получить позиционные эмбеддинги (pos_emb) из таблицы позиционных эмбеддингов.
        Объединить эмбеддинги токенов и позиционные эмбеддинги, добавляя их друг к другу, и сохранить результат в тензор x.
        Проходить тензор x через последовательность блоков трансформера (self.blocks).
        Применить слой нормализации (self.ln_f) к выходу из блоков.
        Проходить выход через линейный слой self.lm_head для получения логитов (logits) для каждого токена.
        Если вам предоставлены целевые токены (targets), вам также нужно рассчитать функцию потерь, используя кросс-энтропию (F.cross_entropy)
        между логитами и целевыми токенами.

        Возвращайте logits и loss.
        """
        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        B, T = idx.size()
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is not None:
            B, T, C = logits.size()
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx - это массив (B, T) с индексами в текущем контексте
        for _ in range(max_new_tokens):
            # Обрезаем idx до последних токенов размером n_ctx
            idx_cond = idx[:, -self.n_ctx:]
            # Получаем прогнозы
            logits, loss = self(idx_cond)
            # Сосредотачиваемся только на последнем шаге времени
            logits = logits[:, -1, :]  # становится (B, C)
            # Применяем софтмакс для получения вероятностей
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # Сэмплируем из распределения
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # Добавляем выбранный индекс к текущей последовательности
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


In [ ]:
# Создание экземпляра модели с параметрами, переданными через `config`
model = GPTModel(**config)

# Перенос модели на указанное устройство (например, GPU)
m = model.to(device)

# Создание оптимизатора AdamW для обучения модели с заданной скоростью обучения (learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])

# Вывод количества параметров в модели в миллионах (1e6)
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

0.209729 M parameters


In [ ]:
max_iters = 5000  # Количество итераций обучения

# Запуск цикла обучения на заданное количество итераций
for iter in range(max_iters):

    # Периодически оцениваем потери на обучающем и валидационном наборах данных
    if iter % config["eval_interval"] == 0 or iter == max_iters - 1:
        # Оценка потерь путем вызова функции estimate_loss с использованием модели
        losses = estimate_loss(model)
        # Выводим информацию о потерях на обучающем и валидационном наборах данных
        print(f"шаг {iter}: потери на обучении {losses['train']:.4f}, потери на валидации {losses['val']:.4f}")

    # Получаем пакет данных (xb - входные данные, yb - целевые значения)
    xb, yb = get_batch('train')

    # Вычисляем логиты и потери, вызывая модель с входными данными
    logits, loss = model(idx=xb, targets=yb)

    # Обнуляем градиенты оптимизатора с помощью optimizer.zero_grad(set_to_none=True)
    optimizer.zero_grad(set_to_none=True)

    # Вычисляем градиенты потерь относительно параметров модели
    loss.backward()

    # Обновляем параметры модели с использованием оптимизатора
    optimizer.step()

шаг 0: потери на обучении 4.3297, потери на валидации 4.3325
шаг 100: потери на обучении 2.6701, потери на валидации 2.6749
шаг 200: потери на обучении 2.4939, потери на валидации 2.5115
шаг 300: потери на обучении 2.4181, потери на валидации 2.4278
шаг 400: потери на обучении 2.3582, потери на валидации 2.3705
шаг 500: потери на обучении 2.3099, потери на валидации 2.3153
шаг 600: потери на обучении 2.2536, потери на валидации 2.2802
шаг 700: потери на обучении 2.2126, потери на валидации 2.2269
шаг 800: потери на обучении 2.1705, потери на валидации 2.1926
шаг 900: потери на обучении 2.1088, потери на валидации 2.1478
шаг 1000: потери на обучении 2.0844, потери на валидации 2.1194
шаг 1100: потери на обучении 2.0611, потери на валидации 2.1002
шаг 1200: потери на обучении 2.0308, потери на валидации 2.1064
шаг 1300: потери на обучении 1.9945, потери на валидации 2.0599
шаг 1400: потери на обучении 1.9861, потери на валидации 2.0538
шаг 1500: потери на обучении 1.9509, потери на валид